<a href="https://colab.research.google.com/github/RanggaPD/Undergraduate-Thesis/blob/main/Declustering_Knopoff_Gardner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Fungsi untuk menghitung jarak waktu antara dua gempa
def calculate_time_difference(time1, time2):
    return abs((time2 - time1).days)

In [ ]:
# Fungsi untuk mendapatkan jendela waktu dan spasial berdasarkan magnitudo
def get_windows(magnitude):
    # Menghitung jendela spasial
    spatial_window = 10**(0.1238 * magnitude + 0.983)
    # Menghitung jendela waktu
    if magnitude >= 6.5:
        time_window = 10**(0.032 * magnitude + 2.7389)
    else:
        time_window = 10**(0.5409 * magnitude - 0.547)
    return spatial_window, time_window

In [ ]:
# Fungsi untuk melakukan declustering
def decluster_earthquake_data(df):
    main_shocks = []
    after_shocks = []

    for i, quake in df.iterrows():
        if i == 0:
            main_shocks.append(quake)
        else:
            is_aftershock = False
            for main_quake in main_shocks:
                spatial_window, time_window = get_windows(main_quake['MAG'])
                time_diff = calculate_time_difference(main_quake['DATE'], quake['DATE'])
                spatial_dist = calculate_spatial_distance(main_quake['LAT'], main_quake['LON'], quake['LAT'], quake['LON'])

                if time_diff <= time_window and spatial_dist <= spatial_window:
                    after_shocks.append(quake)
                    is_aftershock = True
                    break

            if not is_aftershock:
                main_shocks.append(quake)

    return pd.DataFrame(main_shocks), pd.DataFrame(after_shocks)

In [ ]:
# Membaca data gempa dari file
data = pd.read_excel('/content/Data_Setelah_Declustering_Not_Fix.xlsx', parse_dates=['DATE'])

In [ ]:
# Melakukan declustering
main_shocks, after_shocks = decluster_earthquake_data(data)

In [ ]:
# Menyimpan hasil ke file CSV
#main_shocks.to_csv('main_shocks_Fix_1.csv', index=False)
#after_shocks.to_csv('after_shocks_Fix_1.csv', index=False)

#print("Declustering selesai. Data gempa utama dan gempa susulan telah disimpan ke file CSV.")

Declustering selesai. Data gempa utama dan gempa susulan telah disimpan ke file CSV.


In [ ]:
main_shocks_df = pd.DataFrame(main_shocks)
after_shocks_df = pd.DataFrame(after_shocks)

# Menambahkan kolom jenis gempa
main_shocks_df['type'] = 'mainshock'
after_shocks_df['type'] = 'aftershock'

In [ ]:
# Menggabungkan kedua DataFrame
combined_df = pd.concat([main_shocks_df, after_shocks_df]).sort_values(by='DATE').reset_index(drop=True)

# Menyimpan hasil ke file CSV
combined_df.to_csv('combined_shocks_1.csv', index=False)